In [1]:
import pandas as pd
import xgboost as xgb
pd.options.plotting.backend = "plotly"

In [2]:
df = pd.read_csv("google.csv")

In [3]:
df['30DayFwd'] = (df['Close'].shift(-30) - df['Close']) / df['Close']

In [4]:
X              =  df[['Volume']].pct_change()
X['5DayVol']   =  df['Volume'].pct_change().rolling(5).mean().values
X['30DayVol']  =  df['Volume'].pct_change().rolling(30).mean().values
X['252DayVol'] =  df['Volume'].pct_change().rolling(252).mean().values

In [5]:
X['Close30DRatio']  = df['Close'] / df['Close'].rolling(30).mean().values
X['Close60DRatio']  = df['Close'] / df['Close'].rolling(60).mean().values
X['Close252DRatio'] = df['Close'] / df['Close'].rolling(252).mean().values

In [6]:
X['CloseChange']      = df['Close'].pct_change()
X['Close5DayChange']  = df['Close'].pct_change().rolling(5).mean().values
X['Close10DayChange'] = df['Close'].pct_change().rolling(10).mean().values
X['Close60DayChange'] = df['Close'].pct_change().rolling(60).mean().values

In [7]:
 y = df['30DayFwd']

In [8]:
y = y > 0.2

In [9]:
y.mean()

0.060542797494780795

In [10]:
y2 = df['30DayFwd'] < 0

In [11]:
mod = xgb.XGBClassifier()

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, stratify = y_train)

In [14]:
mod.fit(X_train, y_train)

C:\Users\Jonat\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:48:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
X_test

,Volume,5DayVol,30DayVol,252DayVol,Close30DRatio,Close60DRatio,Close252DRatio,CloseChange,Close5DayChange,Close10DayChange,Close60DayChange
54,0.037505,0.038554,0.125802,NaN,1.201757,NaN,NaN,-0.036365,-0.008838,0.022940,NaN
2199,1.523840,0.226267,0.089297,0.084012,1.115991,1.122172,1.293937,0.032454,0.009574,0.011131,0.002192
3271,0.139574,0.008646,0.030488,0.063048,0.988503,0.980719,1.100232,0.005140,0.000924,-0.000341,-0.000202
1580,-0.452888,-0.103391,0.003804,0.070233,0.969983,1.055959,1.099931,-0.008353,-0.002134,-0.004388,0.004305
3896,-0.053032,-0.158647,0.052499,0.062627,1.047842,1.089346,1.227737,0.000073,0.008430,0.003947,0.002573
...,...,...,...,...,...,...,...,...,...,...,...
1433,-0.101935,0.078367,0.103621,0.052037,0.950867,0.962167,1.045325,0.005310,-0.005515,-0.010893,0.000140
3493,0.570987,0.021908,0.074203,0.053503,1.001057,1.031574,1.086042,0.019385,0.003748,-0.003869,0.001535
1081,0.011148,0.050709,0.052317,0.057517,0.889081,0.787258,0.566137,0.015703,-0.000699,-0.004742,-0.005525
2580,0.338491,0.101577,0.060350,0.253244,0.987418,0.955752,0.954370,-0.014493,-0.004023,-0.003393,-0.001302


In [16]:
pd.DataFrame({'Imp': mod.feature_importances_, 
              'Col': X_train.columns})

,Imp,Col
0,0.047454,Volume
1,0.046975,5DayVol
2,0.086782,30DayVol
3,0.109845,252DayVol
4,0.139435,Close30DRatio
5,0.105372,Close60DRatio
6,0.174445,Close252DRatio
7,0.036517,CloseChange
8,0.045962,Close5DayChange
9,0.093354,Close10DayChange


In [17]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(y_test, mod.predict(X_test)))

              precision    recall  f1-score   support

       False       0.97      1.00      0.99       811
        True       0.94      0.58      0.71        52

    accuracy                           0.97       863
   macro avg       0.96      0.79      0.85       863
weighted avg       0.97      0.97      0.97       863



In [19]:
mod.set_params(scale_pos_weight = 1 / y_train.mean())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=16.51497005988024,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [20]:
mod.fit(X_train, y_train)

[11:48:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Jonat\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=16.51497005988024,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [21]:
print(classification_report(y_test, mod.predict(X_test)))

              precision    recall  f1-score   support

       False       0.98      1.00      0.99       811
        True       0.90      0.67      0.77        52

    accuracy                           0.98       863
   macro avg       0.94      0.83      0.88       863
weighted avg       0.97      0.98      0.97       863

